In [ ]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import MonthEnd
import datetime
from dateutil.relativedelta import relativedelta
import calendar
from datetime import date
class Kiddi():
    def __init__(self, IS_RAW_PATH, IWP_PATH, IWES_PATH, CENCUS_PATH):
        self.df = pd.read_excel(IS_RAW_PATH)
        self.df_iwp = pd.read_excel(IWP_PATH)[['idd','peritonitisdate']]
        self.df_iwes = pd.read_excel(IWES_PATH)[['idd','exitsitedate']]
        self.df_cencus = pd.read_excel(CENCUS_PATH)[['idd','PatientAge','male','Diabetes','PDVintage','MQexist']]
        self.target_df = None
        self.population = None
        self.is_raw_partition = dict()
        self.st_data_dt = None
        self.end_data_dt = None
        
    def transform_is_raw(self):
        recentLabCol = ['qRecentLabDt1','qRecentLabDt2','qRecentLabDt3','qRecentLabDt4']
        # Change recent lab date to the end of month
        df = self.get_df()
        for index, col in enumerate(recentLabCol):
          df['ft_data_dt_'+str(index)] = pd.to_datetime(df[col], format="%Y%m")
        feature_list = ["qSodium1Val", "qSodium2Val", "qSodium3Val", "qSodium4Val", "qPotass1Val", "qPotass2Val", "qPotass3Val", 
"qPotass4Val", "qBicarb1Val", "qBicarb2Val", "qBicarb3Val", "qBicarb4Val", "qCreat1Val", "qCreat2Val", "qCreat3Val", 
"qCreat4Val", "qCaTotal1Val", "qCaTotal2Val", "qCaTotal3Val", "qCaTotal4Val", "qPhosph1Val", "qPhosph2Val", "qPhosph3Val", 
"qPhosph4Val", "qFGF231Val", "qFGF232Val", "qFGF233Val", "qFGF234Val", "qMagnesium1Val", "qMagnesium2Val", "qMagnesium3Val", 
"qMagnesium4Val", "qTotalProtein1Val", "qTotalProtein2Val", "qTotalProtein3Val", "qTotalProtein4Val", "qAlbumin1Val", "qAlbumin2Val", "qAlbumin3Val", 
"qAlbumin4Val", "qPTH1Val", "qPTH2Val", "qPTH3Val", "qPTH4Val", "qVitD25OH1Val", "qVitD25OH2Val", "qVitD25OH3Val", 
"qVitD25OH4Val", "qVitD125OH1Val", "qVitD125OH2Val", "qVitD125OH3Val", "qVitD125OH4Val", "qAlkPhosph1Val", "qAlkPhosph2Val", "qAlkPhosph3Val", 
"qAlkPhosph4Val", "qAlkPhosphBone1Val", "qAlkPhosphBone2Val", "qAlkPhosphBone3Val", "qAlkPhosphBone4Val", "qAST1Val", "qAST2Val", "qAST3Val", 
"qAST4Val", "qALT1Val", "qALT2Val", "qALT3Val", "qALT4Val", "qBilir1Val", "qBilir2Val", "qBilir3Val", 
"qBilir4Val", "qUricAcid1Val", "qUricAcid2Val", "qUricAcid3Val", "qUricAcid4Val", "qGlucose1Val", "qGlucose2Val", "qGlucose3Val", 
"qGlucose4Val", "qTotChol1Val", "qTotChol2Val", "qTotChol3Val", "qTotChol4Val", "qHDLChol1Val", "qHDLChol2Val", "qHDLChol3Val", 
"qHDLChol4Val", "qLDLChol1Val", "qLDLChol2Val", "qLDLChol3Val", "qLDLChol4Val", "qTriglyc1Val", "qTriglyc2Val", "qTriglyc3Val", 
"qTriglyc4Val", "qGlycoHgb1Val", "qGlycoHgb2Val", "qGlycoHgb3Val", "qGlycoHgb4Val", "qGlycoAlbumin1Val", "qGlycoAlbumin2Val", "qGlycoAlbumin3Val", 
"qGlycoAlbumin4Val", "qHgb1Val", "qHgb2Val", "qHgb3Val", "qHgb4Val", "qHct1Val", "qHct2Val", "qHct3Val", 
"qHct4Val", "qMCV1Val", "qMCV2Val", "qMCV3Val", "qMCV4Val", "qWhBldCnt1Val", "qWhBldCnt2Val", "qWhBldCnt3Val", 
"qWhBldCnt4Val", "qNeutrophils1Val", "qNeutrophils2Val", "qNeutrophils3Val", "qNeutrophils4Val", "qLymph1Val", "qLymph2Val", "qLymph3Val", 
"qLymph4Val", "qPlatelets1Val", "qPlatelets2Val", "qPlatelets3Val", "qPlatelets4Val", "qTSAT1Val", "qTSAT2Val", "qTSAT3Val", 
"qTSAT4Val", "qIron1Val", "qIron2Val", "qIron3Val", "qIron4Val", "qTIBC1Val", "qTIBC2Val", "qTIBC3Val", 
"qTIBC4Val", "qFerritin1Val", "qFerritin2Val", "qFerritin3Val", "qFerritin4Val", "qCHr1Val", "qCHr2Val", "qCHr3Val", 
"qCHr4Val", "qPctHypoRBC1Val", "qPctHypoRBC2Val", "qPctHypoRBC3Val", "qPctHypoRBC4Val", "qCReactProt1Val", "qCReactProt2Val", "qCReactProt3Val", 
"qCReactProt4Val", "qFibrinogen1Val", "qFibrinogen2Val", "qFibrinogen3Val", "qFibrinogen4Val", "qInterleukin1Val", "qInterleukin2Val", "qInterleukin3Val", 
"qInterleukin4Val", "qBeta2Microglob1Val", "qBeta2Microglob2Val", "qBeta2Microglob3Val", "qBeta2Microglob4Val", "qBNP1Val", "qBNP2Val", "qBNP3Val", 
"qBNP4Val", "qNTproBNP1Val", "qNTproBNP2Val", "qNTproBNP3Val", "qNTproBNP4Val", "qTropT1Val", "qTropT2Val", "qTropT3Val", 
"qTropT4Val", "qTropI1Val", "qTropI2Val", "qTropI3Val", "qTropI4Val", "qHepBAntigen1", "qHepBAntigen2", "qHepBAntigen3", 
"qHepBAntigen4", "qHepBSurfAntiBody1", "qHepBSurfAntiBody2", "qHepBSurfAntiBody3", "qHepBSurfAntiBody4", "qHepBSurfABLvl1", "qHepBSurfABUnit1", "qHepBSurfABLvl2", 
"qHepBSurfABUnit2", "qHepBSurfABLvl3", "qHepBSurfABUnit3", "qHepBSurfABLvl4", "qHepBSurfABUnit4", "qHepBCoreAntibody1", "qHepBCoreAntibody2", "qHepBCoreAntibody3", 
"qHepBCoreAntibody4", "qHepCAntiBody1", "qHepCAntiBody2", "qHepCAntiBody3", "qHepCAntiBody4", "qRecentLabVitalDt1", "qRecentLabVitalDt2", "qRecentLabVitalDt3", 
"qRecentLabVitalDt4", "qDiastolic1", "qSystolic1", "qDiastolic2", "qSystolic2", "qDiastolic3", "qSystolic3", "qDiastolic4", 
"qSystolic4", "qPulse1Val", "qPulse2Val", "qPulse3Val", "qPulse4Val", "qWeightVital1Val", "qWeightVital2Val", "qWeightVital3Val", 
"qWeightVital4Val", "qWeightVolVital1Val", "qWeightVolVital2Val", "qWeightVolVital3Val", "qWeightVolVital4Val", "qExchCAPD1", "qExchCAPD2", "qExchCAPD3", 
"qExchCAPD4", "qCAPDConnSys1", "qCAPDConnSysSpecify1", "qCAPDConnSys2", "qCAPDConnSysSpecify2", "qCAPDConnSys3", "qCAPDConnSysSpecify3", "qCAPDConnSys4", 
"qCAPDConnSysSpecify4", "qExchanges1", "qExchangesSpecify1", "qExchanges2", "qExchangesSpecify2", "qExchanges3", "qExchangesSpecify3", "qExchanges4", 
"qExchangesSpecify4", "qExch1CAPDDwellVol1", "qExch1CAPDDwellVol2", "qExch1CAPDDwellVol3", "qExch1CAPDDwellVol4", "qExch1CAPDBaxSolType1", "qExch1CAPDBaxSolType2", "qExch1CAPDBaxSolType3", 
"qExch1CAPDBaxSolType4", "qExch1CAPDBaxSolLoc", "qExch1CAPDOtherSolType1", "qExch1CAPDOtherSolType2", "qExch1CAPDOtherSolType3", "qExch1CAPDOtherSolType4", "qExch1CAPDOtherSolLoc", "qExch1CAPDDwellTime1", 
"qExch1CAPDDwellTime2", "qExch1CAPDDwellTime3", "qExch1CAPDDwellTime4", "qExch2CAPDDwellVol1", "qExch2CAPDDwellVol2", "qExch2CAPDDwellVol3", "qExch2CAPDDwellVol4", "qExch2CAPDBaxSolType1", 
"qExch2CAPDBaxSolType2", "qExch2CAPDBaxSolType3", "qExch2CAPDBaxSolType4", "qExch2CAPDBaxSolLoc", "qExch2CAPDOtherSolType1", "qExch2CAPDOtherSolType2", "qExch2CAPDOtherSolType3", "qExch2CAPDOtherSolType4", 
"qExch2CAPDOtherSolLoc", "qExch2CAPDDwellTime1", "qExch2CAPDDwellTime2", "qExch2CAPDDwellTime3", "qExch2CAPDDwellTime4", "qExch3CAPDDwellVol1", "qExch3CAPDDwellVol2", "qExch3CAPDDwellVol3", 
"qExch3CAPDDwellVol4", "qExch3CAPDBaxSolType1", "qExch3CAPDBaxSolType2", "qExch3CAPDBaxSolType3", "qExch3CAPDBaxSolType4", "qExch3CAPDBaxSolLoc", "qExch3CAPDOtherSolType1", "qExch3CAPDOtherSolType2", 
"qExch3CAPDOtherSolType3", "qExch3CAPDOtherSolType4", "qExch3CAPDOtherSolLoc", "qExch3CAPDDwellTime1", "qExch3CAPDDwellTime2", "qExch3CAPDDwellTime3", "qExch3CAPDDwellTime4", "qExch4CAPDDwellVol1", 
"qExch4CAPDDwellVol2", "qExch4CAPDDwellVol3", "qExch4CAPDDwellVol4", "qExch4CAPDBaxSolType1", "qExch4CAPDBaxSolType2", "qExch4CAPDBaxSolType3", "qExch4CAPDBaxSolType4", "qExch4CAPDBaxSolLoc", 
"qExch4CAPDOtherSolType1", "qExch4CAPDOtherSolType2", "qExch4CAPDOtherSolType3", "qExch4CAPDOtherSolType4", "qExch4CAPDOtherSolLoc", "qExch4CAPDDwellTime1", "qExch4CAPDDwellTime2", "qExch4CAPDDwellTime3", 
"qExch4CAPDDwellTime4", "qExchLCAPDDwellVol1", "qExchLCAPDDwellVol2", "qExchLCAPDDwellVol3", "qExchLCAPDDwellVol4", "qExchLCAPDBaxSolType1", "qExchLCAPDBaxSolType2", "qExchLCAPDBaxSolType3", 
"qExchLCAPDBaxSolType4", "qExchLCAPDBaxSolLoc", "qExchLCAPDOtherSolType1", "qExchLCAPDOtherSolType2", "qExchLCAPDOtherSolType3", "qExchLCAPDOtherSolType4", "qExchLCAPDOtherSolLoc", "qExchLCAPDDwellTime1", 
"qExchLCAPDDwellTime2", "qExchLCAPDDwellTime3", "qExchLCAPDDwellTime4", "qCAPDPrescrDays1", "qCAPDPrescrDays2", "qCAPDPrescrDays3", "qCAPDPrescrDays4", "qExchAPD1", 
"qExchAPD2", "qExchAPD3", "qExchAPD4", "qAPDCyclerMType1", "qAPDCyclerMTypeSpecify1", "qAPDCyclerMType2", "qAPDCyclerMTypeSpecify2", "qAPDCyclerMType3", 
"qAPDCyclerMTypeSpecify3", "qAPDCyclerMType4", "qAPDCyclerMTypeSpecify4", "qAPDCyclerCSys1", "qAPDCyclerCSys2", "qAPDCyclerCSys3", "qAPDCyclerCSys4", "qNumCycles1", 
"qNumCycles2", "qNumCycles3", "qNumCycles4", "qAPDTotCycleTime1", "qAPDTotCycleTime2", "qAPDTotCycleTime3", "qAPDTotCycleTime4", "qAPDDwellVol1", 
"qAPDDwellVol2", "qAPDDwellVol3", "qAPDDwellVol4", "qTotCyclerVol1", "qTotCyclerVol2", "qTotCyclerVol3", "qTotCyclerVol4", "qCyclerAPDBaxSolType1", 
"qCyclerAPDBaxSolType2", "qCyclerAPDBaxSolType3", "qCyclerAPDBaxSolType4", "qCyclerAPDOtherSolType1", "qCyclerAPDOtherSolType2", "qCyclerAPDOtherSolType3", "qCyclerAPDOtherSolType4", "qTidalAPD1", 
"qTidalAPD2", "qTidalAPD3", "qTidalAPD4", "qTidalAPDPct1", "qTidalAPDPct2", "qTidalAPDPct3", "qTidalAPDPct4", "qAPDNumDayExchange1", 
"qAPDNumDayExchange2", "qAPDNumDayExchange3", "qAPDNumDayExchange4", "qExch1APDCAPDDwellVol1", "qExch1APDCAPDDwellVol2", "qExch1APDCAPDDwellVol3", "qExch1APDCAPDDwellVol4", "qExch1APDCAPDBaxSolType1", 
"qExch1APDCAPDBaxSolType2", "qExch1APDCAPDBaxSolType3", "qExch1APDCAPDBaxSolType4", "qExch1APDCAPDBaxSolLoc", "qExch1APDCAPDOtherSolType1", "qExch1APDCAPDOtherSolType2", "qExch1APDCAPDOtherSolType3", "qExch1APDCAPDOtherSolType4", 
"qExch1APDCAPDOtherSolLoc", "qExch1APDCAPDDwellTime1", "qExch1APDCAPDDwellTime2", "qExch1APDCAPDDwellTime3", "qExch1APDCAPDDwellTime4", "qExch2APDCAPDDwellVol1", "qExch2APDCAPDDwellVol2", "qExch2APDCAPDDwellVol3", 
"qExch2APDCAPDDwellVol4", "qExch2APDCAPDBaxSolType1", "qExch2APDCAPDBaxSolType2", "qExch2APDCAPDBaxSolType3", "qExch2APDCAPDBaxSolType4", "qExch2APDCAPDBaxSolLoc", "qExch2APDCAPDOtherSolType1", "qExch2APDCAPDOtherSolType2", 
"qExch2APDCAPDOtherSolType3", "qExch2APDCAPDOtherSolType4", "qExch2APDCAPDOtherSolLoc", "qExch2APDCAPDDwellTime1", "qExch2APDCAPDDwellTime2", "qExch2APDCAPDDwellTime3", "qExch2APDCAPDDwellTime4", "qExch3APDCAPDDwellVol1", 
"qExch3APDCAPDDwellVol2", "qExch3APDCAPDDwellVol3", "qExch3APDCAPDDwellVol4", "qExch3APDCAPDBaxSolType1", "qExch3APDCAPDBaxSolType2", "qExch3APDCAPDBaxSolType3", "qExch3APDCAPDBaxSolType4", "qExch3APDCAPDBaxSolLoc", 
"qExch3APDCAPDOtherSolType1", "qExch3APDCAPDOtherSolType2", "qExch3APDCAPDOtherSolType3", "qExch3APDCAPDOtherSolType4", "qExch3APDCAPDOtherSolLoc", "qExch3APDCAPDDwellTime1", "qExch3APDCAPDDwellTime2", "qExch3APDCAPDDwellTime3", 
"qExch3APDCAPDDwellTime4", "qExch4APDCAPDDwellVol1", "qExch4APDCAPDDwellVol2", "qExch4APDCAPDDwellVol3", "qExch4APDCAPDDwellVol4", "qExch4APDCAPDBaxSolType1", "qExch4APDCAPDBaxSolType2", "qExch4APDCAPDBaxSolType3", 
"qExch4APDCAPDBaxSolType4", "qExch4APDCAPDBaxSolLoc", "qExch4APDCAPDOtherSolType1", "qExch4APDCAPDOtherSolType2", "qExch4APDCAPDOtherSolType3", "qExch4APDCAPDOtherSolType4", "qExch4APDCAPDOtherSolLoc", "qExch4APDCAPDDwellTime1", 
"qExch4APDCAPDDwellTime2", "qExch4APDCAPDDwellTime3", "qExch4APDCAPDDwellTime4", "qAPDCAPDPrescrDays1", "qAPDCAPDPrescrDays2", "qAPDCAPDPrescrDays3", "qAPDCAPDPrescrDays4", "qCGPDExchange", 
"qCGPrimeDesc", "qCGDescVisit", "qCGWeight", "qCGBP", "qCGSolStrength", "qCGCyclerSetup", "qCGCyclerConn", "qCGCyclerDisConn", 
"qCGExitSiteCare", "qCGCAPDExchange", "qPETestDt1", "qPETestDt2", "qPETestDt3", "qPETestDt4", "qPETestSol1", "qPETestSolSpecify1", 
"qPETestSol2", "qPETestSolSpecify2", "qPETestSol3", "qPETestSolSpecify3", "qPETestSol4", "qPETestSolSpecify4", "qCreatConVal1", "qCreatConUnit1", 
"qCreatConVal2", "qCreatConUnit2", "qCreatConVal3", "qCreatConUnit3", "qCreatConVal4", "qCreatConUnit4", "qCreatCon4HrVal1", "qCreatCon4HrUnit1", 
"qCreatCon4HrVal2", "qCreatCon4HrUnit2", "qCreatCon4HrVal3", "qCreatCon4HrUnit3", "qCreatCon4HrVal4", "qCreatCon4HrUnit4", "qCreatCon4HrCorVal1", "qCreatCon4HrCorUnit1", 
"qCreatCon4HrCorVal2", "qCreatCon4HrCorUnit2", "qCreatCon4HrCorVal3", "qCreatCon4HrCorUnit3", "qCreatCon4HrCorVal4", "qCreatCon4HrCorUnit4", "qDialysGlucConInitVal1", "qDialysGlucConInitUnit1", 
"qDialysGlucConInitVal2", "qDialysGlucConInitUnit2", "qDialysGlucConInitVal3", "qDialysGlucConInitUnit3", "qDialysGlucConInitVal4", "qDialysGlucConInitUnit4", "qDialysGlucCon4HrVal1", "qDialysGlucCon4HrUnit1", 
"qDialysGlucCon4HrVal2", "qDialysGlucCon4HrUnit2", "qDialysGlucCon4HrVal3", "qDialysGlucCon4HrUnit3", "qDialysGlucCon4HrVal4", "qDialysGlucCon4HrUnit4", "qDialysCreatRatio1", "qDialysCreatRatio2", 
"qDialysCreatRatio3", "qDialysCreatRatio4", "qPDDialysInstilled1", "qPDDialysInstilled2", "qPDDialysInstilled3", "qPDDialysInstilled4", "qPDDialysDrained1", "qPDDialysDrained2", 
"qPDDialysDrained3", "qPDDialysDrained4", "qPDSol227Use", "qPDSol386Use", "qRecentLabRKFDt1", "qRecentLabRKFDt2", "qRecentLabRKFDt3", "qRecentLabRKFDt4", 
"qUrine24HrVol1Val", "qUrine24HrVol2Val", "qUrine24HrVol3Val", "qUrine24HrVol4Val", "qUrine24HrCreat1Val", "qUrine24HrCreat2Val", "qUrine24HrCreat3Val", "qUrine24HrCreat4Val", 
"qUrine24HrUrea1Val", "qUrine24HrUrea2Val", "qUrine24HrUrea3Val", "qUrine24HrUrea4Val", "qSerumUrea1Val", "qSerumUrea2Val", "qSerumUrea3Val", "qSerumUrea4Val", 
"qSerumCreat1Val", "qSerumCreat2Val", "qSerumCreat3Val", "qSerumCreat4Val", "qResidKtVUrea1Val", "qResidKtVUrea2Val", "qResidKtVUrea3Val", "qResidKtVUrea4Val", 
"qPeritonealKtVUrea1Val", "qPeritonealKtVUrea2Val", "qPeritonealKtVUrea3Val", "qPeritonealKtVUrea4Val", "qTotKtVUrea1Val", "qTotKtVUrea2Val", "qTotKtVUrea3Val", "qTotKtVUrea4Val", 
"qTot24HrPDFluidIn1Val", "qTot24HrPDFluidIn2Val", "qTot24HrPDFluidIn3Val", "qTot24HrPDFluidIn4Val", "qTot24HrPDFluidOut1Val", "qTot24HrPDFluidOut2Val", "qTot24HrPDFluidOut3Val", "qTot24HrPDFluidOut4Val", 
"qTot24HrDialyUrea1Val", "qTot24HrDialyUrea2Val", "qTot24HrDialyUrea3Val", "qTot24HrDialyUrea4Val", "qCreatClearance1Val", "qCreatClearance2Val", "qCreatClearance3Val", "qCreatClearance4Val", 
"qWeight1Val", "qWeight2Val", "qWeight3Val", "qWeight4Val", "qWeightVol1Val", "qWeightVol2Val", "qWeightVol3Val", "qWeightVol4Val", 
"qSolESC_OSTXT", "qESPStrategy_OSTXT", "qESPOtherOintment", "qPTBloodTransYN4", "qPTBloodTransUnit1", "qPTBloodTransYN3", "qPTBloodTransUnit2", "qPTBloodTransYN2", 
"qPTBloodTransUnit3", "qPTBloodTransYN1", "qPTBloodTransUnit4", "qIndicateLoc1", "qIndicateLoc2", "qIndicateLoc3", "qIndicateLoc4", "qOralNutr", 
"qAminoAcidSuppl", "qVitaminPrep", "qFacilityPDVisit1", "qFacilityPDVisit2", "qFacilityPDVisit3", "qFacilityPDVisit4", "qHomePDVisit1", "qHomePDVisit2", 
"qHomePDVisit3", "qHomePDVisit4", "qHomePDVisitCnt1", "qHomePDVisitCnt2", "qHomePDVisitCnt3", "qHomePDVisitCnt4", "BactInfec1", "BactInfec2", 
"BactInfec3", "BactInfec4", "DialAccessRelInfec1", "DialAccessRelInfec2", "DialAccessRelInfec3", "DialAccessRelInfec4", ]
        rm_key_ft_data_dt = ["idd", "ft_data_dt_0", "ft_data_dt_1", "ft_data_dt_2", "ft_data_dt_3"]
        df = df[rm_key_ft_data_dt + feature_list]
        # rename column
        col1 = [c for c in feature_list if c[-1] == '1' or '1Val' in c]
        col2 = [c for c in feature_list if c[-1] == '2' or '2Val' in c]
        col3 = [c for c in feature_list if c[-1] == '3' or '3Val' in c]
        col4 = [c for c in feature_list if c[-1] == '4' or '4Val' in c]
        col_rename_1 = [(c, c[:-1]) if c[-1] == '1' else (c, c.replace("1Val", "Val")) for c in col1]
        col_rename_2 = [(c, c[:-1]) if c[-1] == '2' else (c, c.replace("2Val", "Val")) for c in col2]
        col_rename_3 = [(c, c[:-1]) if c[-1] == '3' else (c, c.replace("3Val", "Val")) for c in col3]
        col_rename_4 = [(c, c[:-1]) if c[-1] == '4' else (c, c.replace("4Val", "Val")) for c in col4]
        df1 = df[['idd', 'ft_data_dt_0'] + col1].rename(columns = dict(col_rename_1))
        df2 = df[['idd', 'ft_data_dt_1'] + col2].rename(columns = dict(col_rename_2))
        df3 = df[['idd', 'ft_data_dt_2'] + col3].rename(columns = dict(col_rename_3))
        df4 = df[['idd', 'ft_data_dt_3'] + col4].rename(columns = dict(col_rename_4))
        df1 = df1.rename(columns = {'ft_data_dt_0' : 'ft_data_dt'})
        df2 = df2.rename(columns = {'ft_data_dt_1' : 'ft_data_dt'})
        df3 = df3.rename(columns = {'ft_data_dt_2' : 'ft_data_dt'})
        df4 = df4.rename(columns = {'ft_data_dt_3' : 'ft_data_dt'})
        df_new = pd.concat([df1, df2, df3, df4]).sort_values(by=['ft_data_dt', 'idd']).reset_index(drop=True) 
        df_new['ft_data_dt'] = pd.to_datetime(df_new['ft_data_dt'], format="%Y%m") + MonthEnd(1)
        self.set_df(df_new)
        
    def transform_census(self):
        df_census = self.get_df_cencus()
        impute = {'Yes':1, 'No':0, np.nan:np.nan}
        df_census['male'] = df_census['male'].apply(lambda x : impute[x])
        df_census['Diabetes'] = df_census['Diabetes'].apply(lambda x : impute[x])
        self.set_df_census(df_census)
        
    def clean_data(self):
        self.transform_is_raw()
        self.transform_census()
        
    def gen_target(self):
        df_iwp = self.get_df_iwp()
        df_iwes = self.get_df_iwes()
        df_iwes = df_iwes.rename(columns={'exitsitedate':'ft_data_dt'})
        df_iwp = df_iwp.rename(columns={'peritonitisdate':'ft_data_dt'})
        df_infect = df_iwes.merge(df_iwp, how='outer')
        df_infect = df_infect.dropna(axis=0)
        first_infect = df_infect.groupby(['idd']).min()
        first_infect = first_infect.reset_index()
        first_infect['target'] = 1
        first_infect['ft_data_dt'] = pd.to_datetime(first_infect['ft_data_dt'], format="%Y%m") + MonthEnd(1)
        all_target = []
        end_data_dt = self.get_end_data_dt()
        for index, row in first_infect.iterrows():
            date_series = pd.date_range(*(pd.to_datetime([row['ft_data_dt'], end_data_dt]) + pd.offsets.MonthEnd()), freq='M', name='ft_data_dt')
            idd_list = [row['idd']]
            idd_series = pd.Series(data=idd_list, name='idd')
            date_frame = date_series.to_frame()
            idd_frame = idd_series.to_frame()
            date_frame['key'] = 0
            idd_frame['key'] = 0
            tmp_target = idd_frame.merge(date_frame, on='key', how='outer').drop(columns=['key'])
            all_target.append(tmp_target)
        target = pd.concat(all_target)
        target['target'] = 1
        self.set_target_df(target)
        
    def gen_population(self):
        df = self.get_df()
        idd_list = list(set(df['idd'].to_list()))
        date_list = df['ft_data_dt'].to_list()
        st_data_dt = min(date_list)
        end_data_dt = max(date_list)
        self.set_st_data_dt(st_data_dt)
        self.set_end_data_dt(end_data_dt)
        date_series = pd.date_range(*(pd.to_datetime([st_data_dt, end_data_dt]) + pd.offsets.MonthEnd()), freq='M', name='ft_data_dt')
        date_frame = date_series.to_frame()
        idd_series = pd.Series(data=idd_list, name='idd')
        idd_frame = idd_series.to_frame()
        date_frame['key'] = 0
        idd_frame['key'] = 0
        population = idd_frame.merge(date_frame, on='key', how='outer').drop(columns=['key'])
        self.set_population(population)
    
    def partition_is_raw(self):
        is_raw_partition = self.get_is_raw_partition()
        df = self.get_df()
#         df['ft_data_dt'] = pd.to_datetime(df['ft_data_dt'], format="%Y%m") + MonthEnd(1)
        st_data_dt = self.get_st_data_dt()
        end_data_dt = self.get_end_data_dt()
        date_series = pd.date_range(*(pd.to_datetime([st_data_dt, end_data_dt]) + pd.offsets.MonthEnd()), freq='M', name='ft_data_dt')
        date_series = date_series.to_list()
        for d in date_series:
            is_raw_partition[d.strftime("%Y-%m-%d")] = df[df['ft_data_dt'] == d]
        self.set_is_raw_partition(is_raw_partition)
        self.set_df(df)
        
    def feature_engineer(self, month=12):
        st_data_dt = self.get_st_data_dt()
        end_data_dt = self.get_end_data_dt()
        date_list = pd.date_range(*(pd.to_datetime([st_data_dt, end_data_dt]) + pd.offsets.MonthEnd())).to_list()
        population = self.get_population()
        is_raw_partition = self.get_is_raw_partition()
#         Lag 2 months
        population['ft_data_dt'] = population['ft_data_dt'].astype('datetime64[M]') - pd.DateOffset(months=1) + MonthEnd(1)
        all_data = []
#         join past is_raw columns
        for d in date_list:
            population_partition = population[population['ft_data_dt'] == d] 
            old_date = d - relativedelta(months=month)
            y = old_date.year
            m = old_date.month
            day = calendar.monthrange(y, m)[1]
            old_date = date(y, m, day)
            old_date = max(old_date, st_data_dt)
            
            date_list_join = pd.date_range(*(pd.to_datetime([old_date, d]) + pd.offsets.MonthEnd()), freq='M').to_list()
            date_list_join.reverse()
            for index, date_join in enumerate(date_list_join):
                tmp_is_raw_partition = is_raw_partition[date_join.strftime("%Y-%m-%d")]
                rename_col = [c for c in list(tmp_is_raw_partition.columns) if c not in ['idd', 'ft_data_dt']]
                new_col = [c+'_'+str(index+1) for c in rename_col]
                name_dict = dict(list(zip(rename_col, new_col)))
                tmp_is_raw_partition = tmp_is_raw_partition.rename(columns = name_dict)
#                 print(population_partition)
#                 print(tmp_is_raw_partition)
                population_partition = population_partition.merge(tmp_is_raw_partition, on=['idd', 'ft_data_dt'], how='left')
            all_data.append(population_partition)
        
        
        return all_data
        
        
    def get_df(self):
        return self.df
    
    def set_df(self, df):
        self.df = df
        
    def get_df_iwp(self):
        return self.df_iwp
    
    def set_df_iwp(self, df_iwp):
        self.df_iwp = df_iwp
        
    def get_df_iwes(self):
        return self.df_iwes
    
    def set_df_iwes(self, df_iwes):
        self.df_iwes = df_iwes
        
    def get_df_cencus(self):
        return self.df_cencus
    
    def set_df_census(self, df_census):
        self.df_census = df_census
    
    def get_target_df(self):
        return self.target_df
    
    def set_target_df(self, target_df):
        self.target_df = target_df
        
    def get_population(self):
        return self.population
    
    def set_population(self, population):
        self.population = population
        
    def get_st_data_dt(self):
        return self.st_data_dt
    
    def set_st_data_dt(self, st_data_dt):
        self.st_data_dt = st_data_dt
        
    def get_end_data_dt(self):
        return self.end_data_dt
    
    def set_end_data_dt(self, end_data_dt):
        self.end_data_dt = end_data_dt
    
    def get_is_raw_partition(self):
        return self.is_raw_partition
    
    def set_is_raw_partition(self, is_raw_partition):
        self.is_raw_partition = is_raw_partition

In [ ]:
IS_RAW_PATH = '/Users/metis_sotangkur/Desktop/Kiddi_ds/Data/is_raw_idd.xlsx'
IWP_PATH = '/Users/metis_sotangkur/Desktop/Kiddi_ds/Data/iwp_raw_idd.xlsx'
IWES_PATH = '/Users/metis_sotangkur/Desktop/Kiddi_ds/Data/iwes_raw_idd.xlsx'
CENCUS_PATH = '/Users/metis_sotangkur/Desktop/Kiddi_ds/Data/census_idd.xlsx'
kiddi = Kiddi(IS_RAW_PATH, IWP_PATH, IWES_PATH, CENCUS_PATH)
kiddi.clean_data()
kiddi.gen_population()
kiddi.gen_target()
kiddi.partition_is_raw()

In [ ]:
kiddi.feature_engineer()

In [ ]:
pop['ft_data_dt'].astype('datetime64[M]') - pd.DateOffset(months=1) + MonthEnd(1)
